# WLASL100 Training on Google Colab

This notebook trains an ASL recognition model on the 100-sign WLASL dataset using pre-extracted features.

**Prerequisites**: 
- Features extracted using `WLASL100_Feature_Extraction_Colab.ipynb`
- Features saved to Google Drive at `/content/drive/MyDrive/asl_data/extracted_features/`

**Runtime**: Select **GPU** runtime (Runtime → Change runtime type → GPU)

**Estimated time**: 1-2 hours on Colab GPU (vs 30+ hours on laptop)

## Quick Start

This notebook is a simplified training setup for Colab. For the full training implementation, see:
- `scripts/train_phase1.py` - Full training script with all features
- `src/training/stage2_ctc.py` - Stage 2 CTC training

**Note**: This notebook provides a basic training setup. You can also upload and run `scripts/train_phase1.py` directly in Colab by adapting the paths to use Google Drive locations.

## Step 1: Setup - Install Dependencies

In [ ]:
# Install PyTorch and dependencies
!pip install torch torchvision tqdm numpy

# Check GPU availability
import torch
print(f"GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

## Step 2: Mount Google Drive and Load Features

Load the extracted features from Google Drive.

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Load features and vocabulary
import pickle
import json
from pathlib import Path

feature_dir = Path('/content/drive/MyDrive/asl_data/extracted_features')
train_cache = feature_dir / 'features_train_wlasl100.pkl'
val_cache = feature_dir / 'features_val_wlasl100.pkl'
vocab_file = feature_dir / 'vocabulary.json'

# Load cached features
print("Loading cached features...")
with open(train_cache, 'rb') as f:
    train_cache_data = pickle.load(f)

with open(val_cache, 'rb') as f:
    val_cache_data = pickle.load(f)

# Load vocabulary
with open(vocab_file, 'r') as f:
    vocab_data = json.load(f)

gloss_to_id = vocab_data['gloss_to_id']
id_to_gloss = vocab_data['id_to_gloss']
vocab_size = vocab_data['vocab_size']

print(f"\n✓ Loaded features:")
print(f"  Train: {len(train_cache_data)} videos")
print(f"  Val: {len(val_cache_data)} videos")
print(f"  Vocab size: {vocab_size} (100 glosses + blank)")
print(f"  Classes: {len(gloss_to_id)} unique signs")

## Step 3: Define Dataset and Model Architecture

We'll define everything needed for training directly in the notebook (no additional file uploads needed).

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from typing import Dict, List
from tqdm.notebook import tqdm
import numpy as np

# Dataset class for cached features
class CachedFeatureDataset(Dataset):
    """Dataset for cached features."""
    
    def __init__(self, cache_data: Dict):
        """
        Args:
            cache_data: Dict with format {video_id: {'features': ..., 'gloss': ..., 'gloss_id': ...}}
        """
        self.samples = []
        
        for video_id, data in cache_data.items():
            features = data['features']  # (T, 36) numpy array
            gloss_id = data['gloss_id']
            
            # Convert to tensor
            features_tensor = torch.FloatTensor(features)
            
            # Create target sequence (just the gloss_id for CTC)
            target = torch.LongTensor([gloss_id])
            
            self.samples.append({
                'features': features_tensor,
                'target': target,
                'gloss_id': gloss_id,
                'video_id': video_id,
                'length': len(features)
            })
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        return self.samples[idx]


def collate_fn(batch):
    """Collate function for DataLoader - handles variable-length sequences."""
    # Sort by length (descending) for efficient batching
    batch = sorted(batch, key=lambda x: x['length'], reverse=True)
    
    # Get dimensions
    batch_size = len(batch)
    max_length = batch[0]['length']
    feature_dim = batch[0]['features'].shape[1]
    
    # Initialize padded tensors
    features_padded = torch.zeros(batch_size, max_length, feature_dim)
    lengths = torch.LongTensor([s['length'] for s in batch])
    targets = []
    gloss_ids = []
    
    # Fill in data
    for i, sample in enumerate(batch):
        seq_len = sample['length']
        features_padded[i, :seq_len] = sample['features']
        targets.append(sample['target'])
        gloss_ids.append(sample['gloss_id'])
    
    return {
        'features': features_padded,
        'targets': targets,
        'lengths': lengths,
        'gloss_ids': torch.LongTensor(gloss_ids)
    }

print("✓ Dataset class defined")

In [ ]:
# Model Architecture
class ASLEncoder(nn.Module):
    """BiLSTM encoder for ASL features."""
    
    def __init__(self, input_dim=36, hidden_dim=128, num_layers=2, dropout=0.3, bidirectional=True):
        super().__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.bidirectional = bidirectional
        
        self.lstm = nn.LSTM(
            input_dim,
            hidden_dim,
            num_layers,
            dropout=dropout if num_layers > 1 else 0,
            bidirectional=bidirectional,
            batch_first=True
        )
        
        self.output_dim = hidden_dim * 2 if bidirectional else hidden_dim
    
    def forward(self, x, lengths):
        """
        Args:
            x: (batch_size, seq_len, input_dim)
            lengths: (batch_size,) actual sequence lengths
        Returns:
            output: (batch_size, seq_len, output_dim)
        """
        # Pack padded sequence for efficiency
        packed = nn.utils.rnn.pack_padded_sequence(x, lengths.cpu(), batch_first=True, enforce_sorted=False)
        output, _ = self.lstm(packed)
        output, _ = nn.utils.rnn.pad_packed_sequence(output, batch_first=True)
        return output


class CTCModel(nn.Module):
    """CTC model with encoder and classification head."""
    
    def __init__(self, encoder, vocab_size, blank_id=0):
        super().__init__()
        self.encoder = encoder
        self.vocab_size = vocab_size
        self.blank_id = blank_id
        
        # Classification head
        self.classifier = nn.Linear(encoder.output_dim, vocab_size)
    
    def forward(self, x, lengths):
        """
        Args:
            x: (batch_size, seq_len, input_dim)
            lengths: (batch_size,) actual sequence lengths
        Returns:
            logits: (batch_size, seq_len, vocab_size)
        """
        # Encode
        encoded = self.encoder(x, lengths)
        
        # Classify
        logits = self.classifier(encoded)
        
        return logits

print("✓ Model architecture defined")

## Step 4: Training Configuration and Setup

In [ ]:
# Training hyperparameters
HIDDEN_DIM = 128  # Can increase to 256 for better capacity (100 classes)
NUM_LAYERS = 2
DROPOUT = 0.3
BATCH_SIZE = 16  # Adjust based on GPU memory (16GB Colab GPU can handle 16-32)
LEARNING_RATE = 1e-3  # Can lower to 5e-4 for more stable training
NUM_EPOCHS = 50
BLANK_PENALTY = 0.5  # Increased from 0.1 for better blank suppression (ratio-based penalty)
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# Checkpoint directory
CHECKPOINT_DIR = Path('/content/checkpoints')
CHECKPOINT_DIR.mkdir(exist_ok=True)

print(f"{'='*70}")
print(f"Training Configuration")
print(f"{'='*70}")
print(f"Device: {DEVICE}")
if DEVICE == 'cuda':
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
print(f"Hidden dim: {HIDDEN_DIM}")
print(f"Layers: {NUM_LAYERS}")
print(f"Dropout: {DROPOUT}")
print(f"Batch size: {BATCH_SIZE}")
print(f"Learning rate: {LEARNING_RATE}")
print(f"Epochs: {NUM_EPOCHS}")
print(f"Blank penalty: {BLANK_PENALTY}")
print(f"{'='*70}")

# Create datasets
train_dataset = CachedFeatureDataset(train_cache_data)
val_dataset = CachedFeatureDataset(val_cache_data)

# Create data loaders
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    collate_fn=collate_fn,
    num_workers=0
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    collate_fn=collate_fn,
    num_workers=0
)

print(f"\n✓ Datasets and data loaders created")
print(f"  Train: {len(train_dataset)} samples ({len(train_loader)} batches)")
print(f"  Val: {len(val_dataset)} samples ({len(val_loader)} batches)")

## Step 4.5: Load Stage 1 Pre-trained Encoder (Following Paper's Curriculum)

**Important**: The paper's 3-stage curriculum (Section 7.2) requires Stage 1 pre-training before Stage 2 CTC training.

- **Stage 1**: Self-supervised phonological pre-training (learns robust features)
- **Stage 2**: CTC training with pre-trained encoder (fine-tunes for gloss prediction)
- **Stage 3**: WFST fine-tuning (future work)

Loading the Stage 1 checkpoint initializes the encoder with pre-trained features, following the paper's approach.

In [ ]:
# Load Stage 1 pre-trained encoder checkpoint (if available)
# Following paper's 3-stage curriculum: Stage 1 → Stage 2

stage1_checkpoint_path = Path('/content/drive/MyDrive/asl_data/checkpoints/stage1/checkpoint_best.pt')
USE_PRETRAINED = stage1_checkpoint_path.exists()

if USE_PRETRAINED:
    print(f"Loading pre-trained Stage 1 encoder from {stage1_checkpoint_path}...")
    stage1_checkpoint = torch.load(stage1_checkpoint_path, map_location=DEVICE, weights_only=False)
    
    # Check checkpoint structure
    print(f"  Checkpoint keys: {list(stage1_checkpoint.keys())}")
    
    # Extract encoder weights (Stage 1 checkpoint format)
    if 'encoder_state_dict' in stage1_checkpoint:
        encoder_state = stage1_checkpoint['encoder_state_dict']
        print(f"  ✓ Found 'encoder_state_dict' in checkpoint")
    elif 'encoder' in stage1_checkpoint:
        encoder_state = stage1_checkpoint['encoder']
        print(f"  ✓ Found 'encoder' in checkpoint")
    else:
        # Try to load from model_state_dict if it has encoder key
        if 'model_state_dict' in stage1_checkpoint:
            model_state = stage1_checkpoint['model_state_dict']
            # Extract encoder keys (assuming they start with 'encoder.')
            encoder_state = {k.replace('encoder.', ''): v 
                           for k, v in model_state.items() 
                           if k.startswith('encoder.')}
            print(f"  ✓ Extracted encoder from 'model_state_dict'")
        else:
            print("  ⚠️ Checkpoint format not recognized. Available keys:")
            for key in stage1_checkpoint.keys():
                print(f"    - {key}")
            encoder_state = None
            USE_PRETRAINED = False
    
    if encoder_state:
        print(f"\n  Stage 1 Info:")
        print(f"    Epoch: {stage1_checkpoint.get('epoch', 'unknown')}")
        print(f"    Loss: {stage1_checkpoint.get('best_loss', stage1_checkpoint.get('loss', 'unknown')):.4f}")
        print(f"\n  ✓ Stage 1 checkpoint loaded successfully!")
        print(f"  ✓ Will initialize encoder with pre-trained weights")
        print(f"  ✓ Following paper's curriculum: Stage 1 → Stage 2")
    else:
        print("  ⚠️ Could not extract encoder weights from checkpoint")
        USE_PRETRAINED = False
else:
    print(f"⚠️ Stage 1 checkpoint not found at {stage1_checkpoint_path}")
    print(f"  Training will use random initialization (not following paper's curriculum)")
    print(f"  To follow the paper:")
    print(f"    1. Upload checkpoints/stage1/checkpoint_best.pt to Google Drive")
    print(f"    2. Place it at: {stage1_checkpoint_path}")
    print(f"    3. Re-run this cell")
    encoder_state = None

print(f"\n{'='*70}")
print(f"Pre-training Status: {'✓ USING PRE-TRAINED (Following Paper)' if USE_PRETRAINED else '⚠️ RANDOM INIT (Not Following Paper)'}")
print(f"{'='*70}")

In [ ]:
# Create model
encoder = ASLEncoder(
    input_dim=36,
    hidden_dim=HIDDEN_DIM,
    num_layers=NUM_LAYERS,
    dropout=DROPOUT
)

# Load pre-trained encoder weights if available (following paper's curriculum)
if USE_PRETRAINED and encoder_state:
    print(f"\nLoading pre-trained encoder weights into model...")
    try:
        # Try loading with strict=False (allows architecture differences)
        encoder.load_state_dict(encoder_state, strict=False)
        print(f"  ✓ Successfully loaded pre-trained encoder weights!")
        print(f"  ✓ Encoder initialized with Stage 1 pre-trained features")
        print(f"  ✓ Following paper's 3-stage curriculum (Section 7.2)")
    except Exception as e:
        print(f"  ⚠️ Warning: Could not load pre-trained weights: {e}")
        print(f"  Continuing with random initialization...")
else:
    print(f"\n  ⚠️ Using random initialization (Stage 1 checkpoint not loaded)")

model = CTCModel(encoder, vocab_size=vocab_size, blank_id=0)
model = model.to(DEVICE)

# Count parameters
num_params = sum(p.numel() for p in model.parameters())
print(f"\n{'='*70}")
print(f"Model Created")
print(f"{'='*70}")
print(f"Parameters: {num_params:,} (~{num_params/1000:.0f}K)")
print(f"Params/train sample ratio: {num_params/len(train_dataset):.0f}:1")
print(f"Encoder initialization: {'PRE-TRAINED (Stage 1)' if USE_PRETRAINED else 'RANDOM'}")

if num_params / len(train_dataset) > 1000:
    print(f"  ⚠ Warning: High params/sample ratio (>1000:1)")
else:
    print(f"  ✓ Good params/sample ratio (<1000:1)")
print(f"{'='*70}")

# Loss function (CTC)
ctc_loss_fn = nn.CTCLoss(blank=0, reduction='mean', zero_infinity=True)

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-4)

print(f"\n✓ Optimizer created: Adam (lr={LEARNING_RATE})")

## Step 5: Training Loop

In [ ]:
def compute_blank_ratio(logits, blank_id=0):
    """Compute ratio of blank predictions (better than mean-based penalty)."""
    predictions = logits.argmax(dim=-1)
    blank_count = (predictions == blank_id).sum().item()
    total_count = predictions.numel()
    return blank_count / total_count if total_count > 0 else 0.0


def train_epoch(model, train_loader, optimizer, ctc_loss_fn, device, epoch, blank_penalty=0.5):
    """Train for one epoch with improved blank penalty (ratio-based)."""
    model.train()
    total_loss = 0.0
    total_ctc_loss = 0.0
    total_blank_penalty = 0.0
    num_batches = 0
    
    for batch in tqdm(train_loader, desc=f"Epoch {epoch} [Train]"):
        features = batch['features'].to(device)
        targets = batch['targets']
        lengths = batch['lengths']
        gloss_ids = batch['gloss_ids'].to(device)
        
        # Forward pass
        logits = model(features, lengths)
        
        # Prepare CTC inputs
        # logits: (batch_size, seq_len, vocab_size) -> (seq_len, batch_size, vocab_size)
        log_probs = F.log_softmax(logits, dim=-1)
        log_probs = log_probs.transpose(0, 1)  # (seq_len, batch_size, vocab_size)
        
        # Targets for CTC
        target_lengths = torch.LongTensor([len(t) for t in targets]).to(device)
        target_flat = torch.cat(targets).to(device)
        
        # Compute CTC loss
        ctc_loss_value = ctc_loss_fn(log_probs, target_flat, lengths, target_lengths)
        
        # Improved blank penalty: ratio-based (better than mean-based)
        blank_ratio = compute_blank_ratio(logits, blank_id=0)
        blank_penalty_value = blank_penalty * blank_ratio
        loss = ctc_loss_value + blank_penalty_value
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5.0)  # Increased from 1.0
        optimizer.step()
        
        total_loss += loss.item()
        total_ctc_loss += ctc_loss_value.item()
        total_blank_penalty += blank_penalty_value
        num_batches += 1
    
    avg_loss = total_loss / num_batches if num_batches > 0 else 0.0
    avg_ctc_loss = total_ctc_loss / num_batches if num_batches > 0 else 0.0
    avg_blank_penalty = total_blank_penalty / num_batches if num_batches > 0 else 0.0
    
    return avg_loss, avg_ctc_loss, avg_blank_penalty


def validate(model, val_loader, ctc_loss_fn, device, epoch):
    """Validate model."""
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0
    num_batches = 0
    
    with torch.no_grad():
        for batch in tqdm(val_loader, desc=f"Epoch {epoch} [Val]"):
            features = batch['features'].to(device)
            targets = batch['targets']
            lengths = batch['lengths']
            gloss_ids = batch['gloss_ids'].to(device)
            
            # Forward pass
            logits = model(features, lengths)
            
            # Compute loss
            log_probs = F.log_softmax(logits, dim=-1)
            log_probs = log_probs.transpose(0, 1)
            
            target_lengths = torch.LongTensor([len(t) for t in targets]).to(device)
            target_flat = torch.cat(targets).to(device)
            
            loss = ctc_loss_fn(log_probs, target_flat, lengths, target_lengths)
            total_loss += loss.item()
            
            # Greedy decoding (simple accuracy check)
            predictions = logits.argmax(dim=-1)  # (batch_size, seq_len)
            
            # For each sample, get most common non-blank prediction
            for i in range(len(gloss_ids)):
                pred_seq = predictions[i, :lengths[i]].cpu().numpy()
                # Remove blanks and get most common
                non_blank = pred_seq[pred_seq != 0]
                if len(non_blank) > 0:
                    pred_id = np.bincount(non_blank).argmax()
                else:
                    pred_id = 0  # All blanks
                
                if pred_id == gloss_ids[i].item():
                    correct += 1
                total += 1
            
            num_batches += 1
    
    avg_loss = total_loss / num_batches if num_batches > 0 else 0.0
    accuracy = (correct / total * 100) if total > 0 else 0.0
    
    return avg_loss, accuracy

print("✓ Training functions defined")

In [ ]:
# Training loop
print(f"\n{'='*70}")
print(f"Starting Training")
print(f"{'='*70}\n")

best_val_acc = 0.0
best_epoch = 0
train_losses = []
val_losses = []
val_accs = []

for epoch in range(1, NUM_EPOCHS + 1):
    # Train (returns tuple: total_loss, ctc_loss, blank_penalty)
    train_loss, train_ctc_loss, train_blank_penalty = train_epoch(model, train_loader, optimizer, ctc_loss_fn, DEVICE, epoch, BLANK_PENALTY)
    train_losses.append(train_loss)
    
    # Validate
    val_loss, val_acc = validate(model, val_loader, ctc_loss_fn, DEVICE, epoch)
    val_losses.append(val_loss)
    val_accs.append(val_acc)
    
    # Print metrics
    print(f"\nEpoch {epoch}/{NUM_EPOCHS}:")
    print(f"  Train Loss: {train_loss:.4f} (CTC: {train_ctc_loss:.4f}, Blank Penalty: {train_blank_penalty:.4f})")
    print(f"  Val Loss: {val_loss:.4f}")
    print(f"  Val Accuracy: {val_acc:.2f}%")
    
    # Save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_epoch = epoch
        
        checkpoint = {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_acc': val_acc,
            'val_loss': val_loss,
            'train_loss': train_loss,
            'vocab_size': vocab_size,
            'gloss_to_id': gloss_to_id,
            'id_to_gloss': id_to_gloss,
            'hidden_dim': HIDDEN_DIM,
            'num_layers': NUM_LAYERS,
            'dropout': DROPOUT
        }
        
        torch.save(checkpoint, CHECKPOINT_DIR / 'checkpoint_best.pt')
        print(f"  ✓ Saved best model (val_acc: {val_acc:.2f}%)")
    
    # Save periodic checkpoints
    if epoch % 10 == 0:
        torch.save(checkpoint, CHECKPOINT_DIR / f'checkpoint_epoch_{epoch}.pt')
    
    print(f"{'='*70}\n")

print(f"\n{'='*70}")
print(f"Training Complete!")
print(f"{'='*70}")
print(f"Best Val Accuracy: {best_val_acc:.2f}% (epoch {best_epoch})")
print(f"Final Val Accuracy: {val_accs[-1]:.2f}%")
print(f"Checkpoints saved to: {CHECKPOINT_DIR}")
print(f"{'='*70}")

## Step 6: Save Results to Google Drive

In [ ]:
# Copy checkpoints to Google Drive
import shutil

drive_checkpoint_dir = Path('/content/drive/MyDrive/asl_data/checkpoints_wlasl100')
drive_checkpoint_dir.mkdir(exist_ok=True)

# Copy all checkpoints
for checkpoint_file in CHECKPOINT_DIR.glob('*.pt'):
    shutil.copy2(checkpoint_file, drive_checkpoint_dir / checkpoint_file.name)
    print(f"✓ Copied {checkpoint_file.name}")

print(f"\n{'='*70}")
print(f"Results saved to Google Drive")
print(f"{'='*70}")
print(f"Checkpoints: {drive_checkpoint_dir}")
print(f"Best model: {drive_checkpoint_dir / 'checkpoint_best.pt'}")
print(f"\nYou can now download these files to your laptop!")

## Analysis: Why Low Accuracy?

The training completed but achieved only 3.57% accuracy (vs 1% random baseline). This suggests the model may be:
1. Collapsing to blank predictions
2. Using incorrect CTC loss format
3. Having issues with target format

Let's check what the model is actually predicting:

In [ ]:
# Diagnostic: Check what the model is predicting
model.eval()
with torch.no_grad():
    # Get a sample batch
    sample_batch = next(iter(val_loader))
    features = sample_batch['features'].to(DEVICE)
    lengths = sample_batch['lengths']
    gloss_ids = sample_batch['gloss_ids']
    
    # Get predictions
    logits = model(features, lengths)
    predictions = logits.argmax(dim=-1)  # (batch_size, seq_len)
    
    # Check blank ratio
    blank_count = (predictions == 0).sum().item()
    total_predictions = predictions.numel()
    blank_ratio = blank_count / total_predictions
    
    print(f"Sample batch analysis:")
    print(f"  Batch size: {len(gloss_ids)}")
    print(f"  Blank predictions: {blank_count}/{total_predictions} ({blank_ratio*100:.1f}%)")
    print(f"  True gloss IDs: {gloss_ids.tolist()[:5]}...")
    print(f"  Predicted (first frame): {predictions[:, 0].cpu().tolist()[:5]}...")
    print(f"  Most common prediction: {torch.bincount(predictions.flatten().cpu()).argmax().item()}")
    
    # Check if model is just predicting one class
    unique_preds = torch.unique(predictions)
    print(f"  Unique predictions in batch: {len(unique_preds)} (out of {vocab_size} possible)")
    
    if blank_ratio > 0.8:
        print(f"\n⚠ WARNING: Model is predicting mostly blanks (>80%)")
        print(f"   This suggests blank collapse - the model learned to always predict blank token.")
    elif len(unique_preds) < 5:
        print(f"\n⚠ WARNING: Model is only predicting {len(unique_preds)} different classes")
        print(f"   This suggests the model collapsed to a few classes.")

## Fix: Improved Training Setup

The issues found:
- 52% blank predictions (too high)
- Only 17/101 classes being predicted
- Model needs better regularization and training

**Solutions**:
1. Increase blank penalty significantly (0.1 → 0.5)
2. Use better blank penalty computation (ratio-based, like working code)
3. Increase model capacity slightly (hidden_dim 128 → 256)
4. Lower learning rate for more stable training
5. More aggressive gradient clipping

Let's create a fixed training script:

In [ ]:
# Improved training function with better blank penalty
def compute_blank_ratio(logits, blank_id=0):
    """Compute ratio of blank predictions."""
    predictions = logits.argmax(dim=-1)
    blank_count = (predictions == blank_id).sum().item()
    total_count = predictions.numel()
    return blank_count / total_count if total_count > 0 else 0.0


def train_epoch_fixed(model, train_loader, optimizer, ctc_loss_fn, device, epoch, blank_penalty=0.5):
    """Improved training function with better blank penalty computation."""
    model.train()
    total_loss = 0.0
    total_ctc_loss = 0.0
    total_blank_penalty = 0.0
    num_batches = 0
    
    for batch in tqdm(train_loader, desc=f"Epoch {epoch} [Train]"):
        features = batch['features'].to(device)
        targets = batch['targets']
        lengths = batch['lengths']
        gloss_ids = batch['gloss_ids'].to(device)
        
        # Forward pass
        logits = model(features, lengths)
        
        # Prepare CTC inputs (PyTorch format: seq_len, batch, vocab)
        log_probs = F.log_softmax(logits, dim=-1)
        log_probs = log_probs.transpose(0, 1)  # (seq_len, batch_size, vocab_size)
        
        # Targets for CTC
        target_lengths = torch.LongTensor([len(t) for t in targets]).to(device)
        target_flat = torch.cat(targets).to(device)
        
        # Compute CTC loss
        ctc_loss_value = ctc_loss_fn(log_probs, target_flat, lengths, target_lengths)
        
        # Better blank penalty: ratio-based (like working code)
        blank_ratio = compute_blank_ratio(logits, blank_id=0)
        blank_penalty_value = blank_penalty * blank_ratio
        
        # Total loss
        loss = ctc_loss_value + blank_penalty_value
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5.0)  # Increased from 1.0
        optimizer.step()
        
        total_loss += loss.item()
        total_ctc_loss += ctc_loss_value.item()
        total_blank_penalty += blank_penalty_value.item()
        num_batches += 1
    
    return {
        'loss': total_loss / num_batches if num_batches > 0 else 0.0,
        'ctc_loss': total_ctc_loss / num_batches if num_batches > 0 else 0.0,
        'blank_penalty': total_blank_penalty / num_batches if num_batches > 0 else 0.0
    }


print("✓ Improved training function created")

## Option: Retrain with Improved Hyperparameters

To retrain with better settings, you would:

1. **Increase blank penalty**: 0.1 → 0.5 (more aggressive blank suppression)
2. **Increase model size**: hidden_dim 128 → 256 (more capacity for 100 classes)
3. **Lower learning rate**: 1e-3 → 5e-4 (more stable training)
4. **Better blank penalty**: Use ratio-based instead of mean-based

**However**, the current model did train and is saved. The 3.57% accuracy suggests:
- Model is learning something (better than 1% random)
- But not well enough (should be 10-30% for this task)
- May need more data per class, better architecture, or longer training

**Recommendations**:
1. **Keep current checkpoints** - They represent a baseline
2. **Try improving** - Use the fixed training function above with better hyperparameters
3. **Analyze** - Check per-class accuracy to see which signs work and which don't

Would you like to:
- **A**: Analyze the current model further (see per-class performance)
- **B**: Retrain with improved hyperparameters (will take 1-2 more hours)
- **C**: Accept baseline and document findings

## ⚠️ IMPORTANT: Are We Following the Paper?

**NO - We're skipping Stage 1 pre-training!**

### Paper's 3-Stage Curriculum (Section 7.2):

**Stage 1**: Self-supervised phonological pre-training
- ✅ Contrastive learning on unlabeled video (temporal coherence)
- ✅ Phonological quantizer training (Product VQ)
- ✅ Loss: L_contrast + L_phon
- ✅ **Purpose**: Learn robust features BEFORE supervised learning

**Stage 2**: End-to-end CTC training (what we're doing)
- ✅ CTC loss for gloss prediction
- ❌ Should use **pre-trained encoder from Stage 1**
- ❌ Should include multi-task loss: L_CTC + λ_seg L_seg + λ_phon L_phon
- ❌ We're training from random initialization!

**Stage 3**: WFST fine-tuning (future work)
- Discriminative lattice training
- Full pipeline with discourse/morphology

### What We Actually Did:

1. ❌ **Skipped Stage 1** - No pre-training
2. ⚠️ **Stage 2 only** - Training CTC from scratch
3. ❌ **No pre-trained encoder** - Random initialization
4. ❌ **Simplified loss** - Only CTC, no multi-task components

### Why This Matters:

The paper's approach expects:
- **Stage 1** trains the encoder to learn good phonological features
- **Stage 2** fine-tunes with pre-trained weights for better initialization
- Without Stage 1, we're training a random encoder on limited labeled data

**This likely explains the poor performance (3.57%)!**

### Solution Options:

**Option A**: Run Stage 1 pre-training first (on all WLASL videos, unlabeled)
- Learn phonological features
- Then run Stage 2 with pre-trained weights
- **Expected**: Better performance (10-30% accuracy)

**Option B**: Continue with current approach (document as baseline)
- Faster to run (skips pre-training)
- But limited by random initialization
- Useful as "no pre-training" baseline

**Option C**: Hybrid - Use existing Stage 1 checkpoint (if available)
- Check if we have `checkpoints/stage1/checkpoint_best.pt`
- Load pre-trained encoder weights
- Fine-tune on 100-sign dataset

**Recommendation**: Stage 1 pre-training is important per the paper. The poor results likely stem from skipping it.

## ✅ Found Stage 1 Checkpoint!

**Location**: `checkpoints/stage1/checkpoint_best.pt`

According to STATUS.md:
- ✅ Stage 1 Pre-training complete
- Loss: 2.03 (contrastive + VQ)
- Pre-trained encoder ready for Stage 2

**Next**: Load this checkpoint to initialize the encoder with pre-trained weights!

## Option: Load Pre-trained Encoder (Following Paper)

To follow the paper's curriculum, we should:
1. Upload the Stage 1 checkpoint to Google Drive
2. Load the pre-trained encoder weights
3. Fine-tune with Stage 2 CTC training

This should improve performance significantly!

In [ ]:
# Load pre-trained Stage 1 encoder
# First, you'll need to upload checkpoints/stage1/checkpoint_best.pt to Google Drive
# Then uncomment and run this:

"""
# Path to Stage 1 checkpoint (after uploading to Drive)
stage1_checkpoint_path = Path('/content/drive/MyDrive/asl_data/checkpoints/stage1/checkpoint_best.pt')

if stage1_checkpoint_path.exists():
    print(f"Loading pre-trained encoder from {stage1_checkpoint_path}...")
    stage1_checkpoint = torch.load(stage1_checkpoint_path, map_location=DEVICE, weights_only=False)
    
    # Check checkpoint structure
    print(f"\nCheckpoint keys: {list(stage1_checkpoint.keys())}")
    
    # Extract encoder weights (Stage 1 checkpoint format)
    if 'encoder_state_dict' in stage1_checkpoint:
        encoder_state = stage1_checkpoint['encoder_state_dict']
    elif 'encoder' in stage1_checkpoint:
        encoder_state = stage1_checkpoint['encoder']
    else:
        # Try to load from model_state_dict if it has encoder key
        if 'model_state_dict' in stage1_checkpoint:
            model_state = stage1_checkpoint['model_state_dict']
            # Extract encoder keys (assuming they start with 'encoder.')
            encoder_state = {k.replace('encoder.', ''): v 
                           for k, v in model_state.items() 
                           if k.startswith('encoder.')}
        else:
            print("⚠️ Checkpoint format not recognized. Available keys:")
            for key in stage1_checkpoint.keys():
                print(f"  - {key}")
            encoder_state = None
    
    if encoder_state:
        # Load encoder weights (matching keys to our encoder)
        encoder.load_state_dict(encoder_state, strict=False)
        print(f"✓ Loaded pre-trained encoder weights!")
        print(f"  Stage 1 epoch: {stage1_checkpoint.get('epoch', 'unknown')}")
        print(f"  Stage 1 loss: {stage1_checkpoint.get('best_loss', stage1_checkpoint.get('loss', 'unknown')):.4f}")
        print(f"\n  ⚠️ Note: Using 'strict=False' because vocab size may differ")
        print(f"  (Stage 1 was on 20 signs, we're training on 100 signs)")
        print(f"\n  ✓ Encoder initialized with pre-trained features from Stage 1")
        print(f"  ✓ Now ready for Stage 2 CTC fine-tuning (following paper!)")
    else:
        print("⚠️ Could not extract encoder weights from checkpoint")
else:
    print(f"⚠️ Stage 1 checkpoint not found at {stage1_checkpoint_path}")
    print(f"   Please upload checkpoints/stage1/checkpoint_best.pt to Google Drive first")
"""

print("✓ Pre-trained encoder loading code ready")
print("\n📋 Instructions:")
print("1. Upload checkpoints/stage1/checkpoint_best.pt from your laptop to Google Drive")
print("2. Place it at: /content/drive/MyDrive/asl_data/checkpoints/stage1/checkpoint_best.pt")
print("3. Uncomment and run the code above to load pre-trained weights")
print("4. Then retrain with the pre-trained encoder!")